In [5]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

from core.generator import weekly
from core.model import Hierarchy
# from transformer import senticnet

plt.style.use('ggplot')

In [7]:
stock    = '0001.HK'
data_dir = '../data/processed'
lexicon  = 'LMFinance'
split    = 0.8

 
XCOLS  = [
        'close', 'volume', 'ma7', 'ma14', 'ma21', 'pct_chg', 'upper', 'mid', 'lower',
        'adx', 'cci', 'mfi', 'rsi7', 'rsi14','rsi21', 'aroondown', 'aroonup', 'macd',
        'macdsignal', 'macdhist', 'ad', 'nart', 'dcperiod', 'dcphase', 'inhpase',
        'quadrature', 'sine', 'leadsine', 'trendmode'
        ]
YCOL = ['pct_chg']



In [8]:
factor    = pd.read_csv(os.path.join(data_dir, 'factors', '%s.csv' % stock))
sentiment = pd.read_csv(os.path.join(data_dir, 'sentiments', lexicon, '%s.csv' % stock))
data    = pd.merge(factor, sentiment, on='date')
data_p  = [p for p in weekly(data, tau)]
date_p  = [dat.index.values.tolist() for dat in data_p]
data_p  = [daily2period(p) for p in data_p]
i_split = int(len(data_p) * split)

# spread features and label.
X_p = np.array([dat[XCOLS] for dat in data_p])
X_p = StandardScaler().fit(X_p[:i_split]).transform(X_p)
y_p = np.array([dat[YCOL] for dat in data_p])

NameError: name 'tau' is not defined

In [3]:
taus = np.arange(1, 8)
n_class = np.arange(2, 10)

score = []
for tau in taus:
    score_ = []
    for n in n_class:
        X = data.weekly(senticnet,tau)[0]
        X = StandardScaler().fit_transform(X)
        clster = Hierarchy(n, method='ward', criterion='maxclust').fit(X)
        score_.append(clster.silhouette_score_)
    score.append(score_)
score = np.array(score).T
fig = plt.figure()
ax = Axes3D(fig)
grid = np.meshgrid(taus, n_class)
ax.plot_surface(grid[0], grid[1], score, rstride=1, cstride=1, cmap='rainbow')
plt.show()
(taus[np.argmax(score) % len(taus)], n_class[np.argmax(score) // len(taus)])

NameError: name 'data' is not defined

In [4]:
def class_change(X, y, center, radius, rank=1):
    change = abs(np.sign(np.append(0, np.diff(y,rank)))) # change[0] === 0
    dist = []
    for i in range(len(change)):
        if change[i] == 0:
            dist.append(0)
        else:
            precenter = center[y[i-rank]]
            preradius = radius[y[i-rank]]
            dist.append(abs((np.linalg.norm(X[i] - precenter) - preradius) / preradius))
    return dist

In [5]:
tau = 4
n_class = np.arange(2,6)

X = data.weekly(senticnet,tau)[0]
X.describe()
X = StandardScaler().fit_transform(X)
fig, ax = plt.subplots(1, 4, figsize=(15, 3), sharey=True)
for i, n in enumerate(n_class):
    cluster = Hierarchy(n, method='ward', criterion='maxclust').fit(X)
    dist = class_change(X, cluster.predict(), cluster.center, cluster.radius)
    ax[i].bar(np.arange(len(dist)), dist)
    ax[i].set_xlabel('Periods', fontsize=10)
    ax[i].set_ylabel('Distance between styles', fontsize=10)
    ax[i].set_title('style number = %d' % n, color=(.3,.3,.3), fontsize=10)
plt.tight_layout()
plt.show()

NameError: name 'data' is not defined

风格的变换有没有周期性：风格自身的周期研究，离散ACD

In [6]:
tau, n = 4, 3
threshold = 0.5

X = data.weekly(senticnet,tau)[0]
X = StandardScaler().fit_transform(X)

cluster = Hierarchy(n, method='ward', criterion='maxclust').fit(X)
center = cluster.center
fig, ax = plt.subplots(1, 2, figsize=(13, 2))
ax[0] = sns.heatmap(center,-4, 4, annot=True, square=True, cmap='RdYlGn', fmt='.2f', xticklabels=cols[1:], ax=ax[0])
ax[0].tick_params(labelsize=10)
# take the sign of each center.
for i in range(center.shape[0]):
    for j in range(center.shape[1]):
        center[i,j] = np.sign(center[i,j]) if abs(center[i,j]) > threshold else 0
ax[1] = sns.heatmap(center,-1, 1, annot=True, square=True, cmap='RdYlGn', fmt='.0f', xticklabels=cols[1:], ax=ax[1])
ax[1].tick_params(labelsize=10)
plt.tight_layout()
plt.show()

NameError: name 'data' is not defined